# Notebook 4: Spatial Statistics with Squidpy
**Tutor:** Anthony Christidis **Time:** 45 minutes

Welcome to spatial statistics analysis! In this notebook, we'll use squidpy to ask sophisticated questions about spatial organization in Visium data.

Spatial statistics go beyond simple visualization - they quantify patterns, test hypotheses, and reveal biological insights about tissue organization.

**Goals:**
1. Identify spatially variable genes using Moran's I spatial autocorrelation
2. Analyze neighborhood enrichment to understand tissue organization  
3. Investigate ligand-receptor interactions between tissue regions
4. Perform spatial co-occurrence analysis

In [ ]:
%load_ext jupyter_black

import spatialdata as sd
import scanpy as sc
import squidpy as sq
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.lines import Line2D
import warnings
warnings.filterwarnings("ignore")

# Print versions for reproducibility
for p in [sd, sc, sq]:
    print(f"{p.__name__}: {p.__version__}")

print("\n=== Loading Visium Glioblastoma Data ===")
sdata_visium = sd.read_zarr("../data/visium_glioblastoma_subset.zarr")
adata_visium = sdata_visium.tables["table"].copy()

print(f"Loaded {adata_visium.n_obs} spots with {adata_visium.n_vars} genes")

## Part 1: Data Preparation and Basic Processing

Before diving into spatial statistics, we need to preprocess our data and run basic clustering to identify tissue regions.

In [ ]:
# Calculate QC metrics
sc.pp.calculate_qc_metrics(adata_visium, percent_top=(20, 50), inplace=True)

# Filter low-quality spots and rare genes
print(f"Starting with: {adata_visium.n_obs} spots")
sc.pp.filter_cells(adata_visium, min_counts=500)
sc.pp.filter_genes(adata_visium, min_cells=10)
print(f"After filtering: {adata_visium.n_obs} spots")

# Standard preprocessing pipeline
sc.pp.normalize_total(adata_visium, inplace=True)
sc.pp.log1p(adata_visium)
sc.pp.highly_variable_genes(adata_visium)
sc.pp.pca(adata_visium, use_highly_variable=True)
sc.pp.neighbors(adata_visium)
sc.tl.leiden(adata_visium, key_added="leiden_clusters")
sc.tl.umap(adata_visium)

n_clusters = len(adata_visium.obs['leiden_clusters'].unique())
print(f"Identified {n_clusters} tissue regions/clusters")

In [ ]:
# Visualize the identified clusters
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# UMAP view
sc.pl.umap(adata_visium, color="leiden_clusters", ax=axes[0], show=False, frameon=False)
axes[0].set_title('Tissue Regions (UMAP)')

# Spatial view using our reliable matplotlib method
coords = adata_visium.obsm['spatial']
x_coords = coords[:, 0]
y_coords = coords[:, 1]
cluster_codes = adata_visium.obs['leiden_clusters'].astype('category').cat.codes

scatter = axes[1].scatter(x_coords, y_coords, c=cluster_codes, 
                         cmap='tab10', s=25, alpha=0.8)
axes[1].set_title('Tissue Regions (Spatial View)')
axes[1].set_aspect('equal')

# Add legend
unique_clusters = sorted(adata_visium.obs['leiden_clusters'].unique())
colors = plt.cm.tab10(np.linspace(0, 1, len(unique_clusters)))
legend_elements = [Line2D([0], [0], marker='o', color='w', 
                         markerfacecolor=colors[i], markersize=8, 
                         label=f'Region {cluster}') 
                  for i, cluster in enumerate(unique_clusters)]
axes[1].legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

## Part 2: Spatial Neighbors Graph

The foundation of spatial analysis is building a graph that connects spatially adjacent spots. This allows us to quantify spatial relationships.

In [ ]:
print("=== Building Spatial Neighbors Graph ===")

# Build spatial neighborhood graph
# This connects each spot to its spatial neighbors
sq.gr.spatial_neighbors(adata_visium)

print(f"Built spatial graph with {adata_visium.obsp['spatial_connectivities'].nnz} connections")
print(f"Average neighbors per spot: {adata_visium.obsp['spatial_connectivities'].nnz / adata_visium.n_obs:.1f}")

## Part 3: Spatially Variable Genes - Moran's I Analysis

**Biological Question:** Which genes show non-random spatial expression patterns?

Moran's I quantifies spatial autocorrelation - genes with high Moran's I are expressed in spatially coherent regions, often marking anatomical structures or functional domains.

In [ ]:
print("=== Moran's I Spatial Autocorrelation Analysis ===")

# Calculate Moran's I for spatially variable gene detection
# We'll test highly variable genes for computational efficiency
hvg_genes = adata_visium.var_names[adata_visium.var['highly_variable']]

sq.gr.spatial_autocorr(
    adata_visium,
    mode="moran",
    genes=hvg_genes,
    n_perms=100,  # Number of permutations for statistical testing
    n_jobs=4      # Parallel processing
)

# Display the top spatially variable genes
moran_results = adata_visium.uns["moranI"].sort_values(by="I", ascending=False)
print("\nTop 10 spatially variable genes:")
print(moran_results.head(10)[['I', 'pval_sim']].round(4))

print("\nBottom 10 spatially variable genes (most random):")
print(moran_results.tail(10)[['I', 'pval_sim']].round(4))

In [ ]:
# Visualize top spatially variable genes
top_genes = moran_results.head(3).index.tolist()
bottom_genes = moran_results.tail(3).index.tolist()
genes_to_plot = top_genes + bottom_genes

fig, axs = plt.subplots(2, 3, figsize=(18, 12))
axs = axs.flatten()

coords = adata_visium.obsm['spatial']
x_coords = coords[:, 0]
y_coords = coords[:, 1]

for idx, gene in enumerate(genes_to_plot):
    moran_score = moran_results.loc[gene, 'I']
    p_value = moran_results.loc[gene, 'pval_sim']
    
    # Get gene expression values
    gene_expr = adata_visium[:, gene].X.toarray().flatten() if hasattr(adata_visium.X, 'toarray') else adata_visium[:, gene].X.flatten()
    
    scatter = axs[idx].scatter(x_coords, y_coords, c=gene_expr, 
                              cmap='viridis', s=20, alpha=0.8)
    axs[idx].set_title(f'{gene}\nMoran\'s I = {moran_score:.3f} (p = {p_value:.3f})')
    axs[idx].set_aspect('equal')
    plt.colorbar(scatter, ax=axs[idx])

plt.suptitle('Spatially Variable Genes\nTop row: High spatial coherence, Bottom row: Random patterns', fontsize=16)
plt.tight_layout()
plt.show()

print("High Moran's I (top row) = genes with spatially coherent expression")
print("Low Moran's I (bottom row) = genes with spatially random expression")

## Part 4: Neighborhood Enrichment Analysis

**Biological Question:** Which tissue regions tend to be spatially adjacent to each other?

This analysis reveals the "social network" of tissue regions, identifying which clusters prefer to be neighbors and which tend to segregate.

In [ ]:
print("=== Neighborhood Enrichment Analysis ===")

# Calculate neighborhood enrichment between tissue regions
sq.gr.nhood_enrichment(adata_visium, cluster_key="leiden_clusters")

# Visualize the enrichment matrix
fig, ax = plt.subplots(figsize=(10, 8))
sq.pl.nhood_enrichment(
    adata_visium, 
    cluster_key="leiden_clusters",
    method="ward",  # Hierarchical clustering to group similar patterns
    cmap="RdBu_r",  # Red-blue colormap (red=enriched, blue=depleted)
    ax=ax
)
plt.title('Neighborhood Enrichment Between Tissue Regions')
plt.show()

print("\nInterpretation:")
print("• Red (positive Z-score): Regions are neighbors more often than expected by chance")
print("• Blue (negative Z-score): Regions avoid each other spatially") 
print("• White (Z-score ≈ 0): Random spatial association")

## Part 5: Co-occurrence Analysis

**Biological Question:** How does the spatial association between regions change with distance?

Co-occurrence analysis quantifies how often different regions are found together across increasing spatial scales.

In [ ]:
print("=== Spatial Co-occurrence Analysis ===")

# Calculate co-occurrence across spatial dimensions
sq.gr.co_occurrence(adata_visium, cluster_key="leiden_clusters")

# Visualize co-occurrence for the most abundant cluster
cluster_counts = adata_visium.obs['leiden_clusters'].value_counts()
most_abundant_cluster = cluster_counts.index[0]

# Don't pass ax parameter - let squidpy handle the plotting
plt.figure(figsize=(12, 6))
sq.pl.co_occurrence(
    adata_visium,
    cluster_key="leiden_clusters", 
    clusters=most_abundant_cluster,
    figsize=(12, 6)  # Use figsize parameter instead of ax
)
plt.suptitle(f'Co-occurrence Analysis: Region {most_abundant_cluster}')
plt.show()

print(f"Analyzed co-occurrence for region {most_abundant_cluster} ({cluster_counts[most_abundant_cluster]} spots)")
print("Co-occurrence score = conditional probability of finding regions together at different distances")

## Part 6: Ligand-Receptor Interaction Analysis

**Biological Question:** What molecular interactions might drive communication between neighboring tissue regions?

This analysis identifies potential ligand-receptor pairs that could mediate cell-cell communication between spatially adjacent regions.

In [ ]:
print("=== Ligand-Receptor Interaction Analysis ===")

# Check if we have enough genes for ligand-receptor analysis
print(f"Available genes in dataset: {adata_visium.n_vars}")
print(f"Unique clusters: {adata_visium.obs['leiden_clusters'].unique()}")

try:
    # Perform ligand-receptor analysis between tissue regions
    sq.gr.ligrec(
        adata_visium,
        n_perms=100,
        cluster_key="leiden_clusters"
    )
    
    print("Ligand-receptor analysis complete!")
    
    # Check available interaction data
    if 'ligrec' in adata_visium.uns:
        lr_results = adata_visium.uns['ligrec']
        print("Found ligand-receptor analysis results")
        
        # Display summary of results
        if hasattr(lr_results, 'shape'):
            print(f"Results shape: {lr_results.shape}")
        elif isinstance(lr_results, dict):
            print(f"Results contain {len(lr_results)} entries")
            
    else:
        print("No ligand-receptor results stored")
        
except Exception as e:
    print(f"Ligand-receptor analysis failed: {str(e)}")
    print("This is common with targeted gene panels that have limited ligand-receptor coverage")
    print("Skipping ligand-receptor visualization...")

In [ ]:
# Visualize ligand-receptor interactions (if available)
if 'ligrec' in adata_visium.uns and len(adata_visium.uns['ligrec']) > 0:
    # Select clusters with enough spots for meaningful analysis
    cluster_counts = adata_visium.obs['leiden_clusters'].value_counts()
    major_clusters = cluster_counts.head(3).index.tolist()
    
    fig, ax = plt.subplots(figsize=(12, 8))
    sq.pl.ligrec(
        adata_visium,
        cluster_key="leiden_clusters",
        source_groups=major_clusters[0],
        target_groups=major_clusters[1:3],
        means_range=(0.3, np.inf),  # Filter lowly expressed genes
        alpha=0.05,  # P-value threshold
        swap_axes=True,
        ax=ax
    )
    plt.title(f'Ligand-Receptor Interactions: {major_clusters[0]} → {major_clusters[1:3]}')
    plt.show()
    
    print(f"Ligand-receptor analysis between regions {major_clusters[0]} and {major_clusters[1:3]}")
else:
    print("Ligand-receptor visualization skipped - insufficient interaction data")
    print("This is common with targeted gene panels that don't include many ligand-receptor pairs")

## Part 7: Interactive Spatial Analysis (Instructor Demo)

**Note:** This section demonstrates interactive analysis using napari. For Docker users, this requires graphics setup, so follow along on the instructor's screen.

In [ ]:
print("=== Interactive Spatial Analysis Demo ===")

# Interactive napari demonstration (instructor will run this live)
print("Instructor will demonstrate:")
print("1. Loading spatial data in napari")
print("2. Overlaying gene expression on tissue images")
print("3. Interactive exploration of spatially variable genes")
print("4. Manual annotation of regions of interest")

# Uncomment for live interactive session:
import napari_spatialdata as nsd
viewer = nsd.Interactive(sdata_visium)
# print("Napari viewer launched - follow along on instructor's screen")

## Part 8: Workshop Summary and Biological Insights

In [ ]:
print("=== Spatial Statistics Workshop Summary ===")

# Calculate summary statistics
n_spatially_variable = (moran_results['pval_sim'] < 0.05).sum()
top_gene = moran_results.index[0]
top_moran = moran_results.iloc[0]['I']

print("\n📊 ANALYSIS SUMMARY:")
print(f"• Dataset: {adata_visium.n_obs} tissue spots, {adata_visium.n_vars} genes")
print(f"• Tissue regions identified: {n_clusters}")
print(f"• Spatial neighbors per spot: {adata_visium.obsp['spatial_connectivities'].nnz / adata_visium.n_obs:.1f}")
print(f"• Spatially variable genes (p<0.05): {n_spatially_variable} out of {len(hvg_genes)} tested")
print(f"• Most spatially coherent gene: {top_gene} (Moran's I = {top_moran:.3f})")

print("\n🔬 SPATIAL STATISTICS METHODS LEARNED:")
print("• Spatial Neighbors Graph: Foundation for all spatial analyses")
print("• Moran's I: Identifies genes with spatial expression patterns") 
print("• Neighborhood Enrichment: Reveals tissue region adjacency preferences")
print("• Co-occurrence Analysis: Quantifies spatial associations across distances")
print("• Ligand-Receptor Analysis: Identifies potential cell communication mechanisms")

print("\n🧬 BIOLOGICAL INSIGHTS:")
print("• Spatial gene expression patterns reveal tissue architecture")
print("• Tissue regions show non-random spatial organization")
print("• Quantitative spatial analysis complements visual inspection")
print("• Molecular mechanisms can be inferred from spatial patterns")

print("\n🚀 NEXT STEPS FOR YOUR RESEARCH:")
print("• Apply these methods to your own spatial datasets")
print("• Compare spatial organization between conditions (healthy vs. disease)")
print("• Validate predicted ligand-receptor interactions with functional studies")
print("• Integrate with single-cell RNA-seq for deeper cellular insights")
print("• Explore advanced methods like spatial domain detection")

print("\n📚 RESOURCES:")
print("• Squidpy documentation: https://squidpy.readthedocs.io/")
print("• SpatialData ecosystem: https://spatialdata.scverse.org/")
print("• scverse community: https://discourse.scverse.org/")

=== Spatial Statistics Workshop Summary ===

📊 ANALYSIS SUMMARY:
• Dataset: 5734 tissue spots, 17010 genes
• Tissue regions identified: 17
• Spatial neighbors per spot: 5.8
• Spatially variable genes (p<0.05): 3057 out of 3762 tested
• Most spatially coherent gene: HBB (Moran's I = 0.814)

🔬 SPATIAL STATISTICS METHODS LEARNED:
• Spatial Neighbors Graph: Foundation for all spatial analyses
• Moran's I: Identifies genes with spatial expression patterns
• Neighborhood Enrichment: Reveals tissue region adjacency preferences
• Co-occurrence Analysis: Quantifies spatial associations across distances
• Ligand-Receptor Analysis: Identifies potential cell communication mechanisms

🧬 BIOLOGICAL INSIGHTS:
• Spatial gene expression patterns reveal tissue architecture
• Tissue regions show non-random spatial organization
• Quantitative spatial analysis complements visual inspection
• Molecular mechanisms can be inferred from spatial patterns

🚀 NEXT STEPS FOR YOUR RESEARCH:
• Apply these methods to